In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from shapely import Point, Polygon
import geopandas as gpd
import folium

In [2]:
from utils_data import get_contours_city_ign, get_iris_contours_city, get_data_carroyee_city

In [3]:
code_insee = '44055'

# Pour avoir les bornes au sein de la ville
file_ign = '../Data/BDCARTO/44_Loire_Atlantique/data.gpkg'
file_bornes = '../data/consolidation-etalab-schema-irve-statique-v-2.3.1-20250129.csv'

# Pour avoir les contours des zones à étudier
file_carroye_200 = '../Data/Filosofi2017_carreaux_200m_met.gpkg'
file_carroye_1km = '../data/carreaux_1km_met.gpkg'
file_iris_contours = '../data/IRIS/contours-iris.gpkg'

# Pour stocker les données
processed_dir = '../data/processed'
os.makedirs(processed_dir, exist_ok=True)
# export_file_name_iris = f"{processed_dir}/cs_iris_{str(code_insee)}.csv" 
export_file_name_iris = f"{processed_dir}/cs_iris_{str(code_insee)}.gpkg" 
# export_file_name_200 = f"{processed_dir}/cs_200_{str(code_insee)}.csv" 
export_file_name_200 = f"{processed_dir}/cs_200_{str(code_insee)}.gpkg" 
# export_file_name_1km = f"{processed_dir}/cs_1km_{str(code_insee)}.csv" 
export_file_name_1km = f"{processed_dir}/cs_1km_{str(code_insee)}.gpkg" 

# Pour faire les calculs et visualiser
crs_meters = 'EPSG:2154'
crs_angles = 'EPSG:4326'

# Identifiant qui permet de savoir quelles bornes appartiennent à Vinci / Easycharge
vinci_names_operateur = ['Easy Charge | FR*ECH', 'Easycharge services']

# Loading et visualisation des bornes dans la ville

In [4]:
df_bornes = pd.read_csv(file_bornes)

# Convert latitude & longitude to a geometry column
df_bornes['geometry'] = df_bornes.apply(lambda row: Point(row['consolidated_longitude'], row['consolidated_latitude']), axis=1)
gdf_bornes = gpd.GeoDataFrame(df_bornes, geometry='geometry')
gdf_bornes.set_crs(crs_angles, inplace=True)
gdf_bornes.head(3)

C:\Users\eliot\AppData\Local\Temp\ipykernel_5972\1686685931.py:1: DtypeWarning: Columns (12,18,19,20,21,22,24,29,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df_bornes = pd.read_csv(file_bornes)


,nom_amenageur,siren_amenageur,contact_amenageur,nom_operateur,contact_operateur,telephone_operateur,nom_enseigne,id_station_itinerance,id_station_local,nom_station,...,datagouv_resource_id,datagouv_organization_or_owner,created_at,consolidated_longitude,consolidated_latitude,consolidated_code_postal,consolidated_commune,consolidated_is_lon_lat_correct,consolidated_is_code_insee_verified,geometry
0,SOLVEO ENERGIES,508886132.0,bornes@solveo-energies.com,SOLVEO ENERGIES,bornes@solveo-energies.com,05 32 98 01 58,SOLVEO ENERGIES,DKMONE3785539,NaN,Golf de Garonne,...,a9352425-5392-4028-b02d-fcc88b856a0e,solveo-energies,2024-12-19T10:53:36.428000+00:00,1.407644,43.628261,NaN,Toulouse,True,True,POINT (1.40764 43.62826)
1,SOLVEO ENERGIES,508886132.0,bornes@solveo-energies.com,SOLVEO ENERGIES,bornes@solveo-energies.com,05 32 98 01 58,SOLVEO ENERGIES,DKMONE3785539,NaN,Golf de Garonne,...,a9352425-5392-4028-b02d-fcc88b856a0e,solveo-energies,2024-12-19T10:53:36.428000+00:00,1.407644,43.628261,NaN,Toulouse,True,True,POINT (1.40764 43.62826)
2,SOLVEO ENERGIES,508886132.0,bornes@solveo-energies.com,SOLVEO ENERGIES,bornes@solveo-energies.com,05 32 98 01 58,SOLVEO ENERGIES,DKMONE3785539,NaN,Golf de Garonne,...,a9352425-5392-4028-b02d-fcc88b856a0e,solveo-energies,2024-12-19T10:53:36.428000+00:00,1.407644,43.628261,NaN,Toulouse,True,True,POINT (1.40764 43.62826)


In [5]:
# Getting data for the considered city
gdf_contours_city, geometry_contours_city = get_contours_city_ign(file_ign, code_insee, crs=crs_angles)
center_lon, center_lat = geometry_contours_city.centroid.coords[0]
gdf_bornes_city = gdf_bornes[gdf_bornes.geometry.intersects(geometry_contours_city)]

# Ajout d'un booléen qui indique si la borne appartient à Vinci ou non
gdf_bornes_city['bornes_vinci'] = gdf_bornes_city['nom_operateur'].isin(vinci_names_operateur).astype(int)
gdf_bornes_city.head()

c:\Users\eliot\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,nom_amenageur,siren_amenageur,contact_amenageur,nom_operateur,contact_operateur,telephone_operateur,nom_enseigne,id_station_itinerance,id_station_local,nom_station,...,datagouv_organization_or_owner,created_at,consolidated_longitude,consolidated_latitude,consolidated_code_postal,consolidated_commune,consolidated_is_lon_lat_correct,consolidated_is_code_insee_verified,geometry,bornes_vinci
12331,ChargeGuru,NaN,info@chargeguru.com,ChargeGuru,info@chargeguru.com,+33176341178,EPAF VACANCES LA BAULE - 44500 - 1,FRCG0E001287,FRCG0E001287,EPAF VACANCES LA BAULE - 44500 - 1,...,eco-movement,2023-06-28T11:46:08.539000+00:00,-2.361614,47.276049,NaN,NaN,False,False,POINT (-2.36161 47.27605),0
12332,RAIDEN SAS (ChargeGuru),NaN,NaN,ChargePoint Austria GmbH | FR*CG0,cpo-tech@chargeguru.com,NaN,ChargeGuru,FRCG0P1208388749112575279,978730,ChargeGuru/3ad12c64-283f-4154-adc0-1ebda5c24410,...,gireve-2,2023-03-24T14:32:54.036000+00:00,-2.361614,47.276049,NaN,NaN,False,False,POINT (-2.36161 47.27605),0
12419,ChargeGuru,NaN,info@chargeguru.com,ChargeGuru,info@chargeguru.com,+33176341178,EPAF VACANCES LA BAULE - 44500 - 1,FRCG0E001466,FRCG0E001466,EPAF VACANCES LA BAULE - 44500 - 1,...,eco-movement,2023-06-28T11:46:08.539000+00:00,-2.361614,47.276049,NaN,NaN,False,False,POINT (-2.36161 47.27605),0
12420,RAIDEN SAS (ChargeGuru),NaN,NaN,ChargePoint Austria GmbH | FR*CG0,cpo-tech@chargeguru.com,NaN,ChargeGuru,FRCG0P1208388749112575279,978730,ChargeGuru/3ad12c64-283f-4154-adc0-1ebda5c24410,...,gireve-2,2023-03-24T14:32:54.036000+00:00,-2.361614,47.276049,NaN,NaN,False,False,POINT (-2.36161 47.27605),0
22926,EasyCharge Services,901772400.0,exploitation@easycharge-vinci.com,Easycharge services,exploitation@easycharge-vinci.COM,3 67 47 06 40,Easycharge services,FRECHPN1QJPHUMBX,N1QJPHUMBX,"La Baule , Pkg Atlantia - Palais des Festivals",...,easycharge-services,2022-12-27T17:28:14.243000+00:00,-2.409128,47.284032,NaN,NaN,False,False,POINT (-2.40913 47.28403),1


In [6]:
# Create a folium map centered on the centroid
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Ajouter le contour de la ville
folium.GeoJson(
    geometry_contours_city,
    name="La Baule",
    style_function=lambda x: {"color": "black", "weight": 2, "fillOpacity": 0.1}
).add_to(m)

for idx, row in gdf_bornes_city.iterrows():
    popup_text = f"""
    <strong>Charging Station</strong><br>
    Nb pdc: {row['nbre_pdc']}<br>
    Amenageur: {row['nom_amenageur']}<br>
    Operateur: {row['nom_operateur']}
    """
    marker_color = 'red' if row['nom_operateur'] in vinci_names_operateur else 'blue'
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=popup_text,
        icon=folium.Icon(color=marker_color)
    ).add_to(m)
# Save the map to an HTML file or display it
# m.save("map_france_data.html")
m

# Calcul du nb de bornes par zone

In [7]:
# FIltrage pour avoir un dataset facile à traiter
cols_to_keep = ['bornes_vinci', 'nbre_pdc', 'puissance_nominale', 'geometry']
gdf_bornes_city = gdf_bornes_city[cols_to_keep]
gdf_bornes_city = gdf_bornes_city.reset_index(drop=True) 
gdf_bornes_city.head()

,bornes_vinci,nbre_pdc,puissance_nominale,geometry
0,0,2,22.0,POINT (-2.36161 47.27605)
1,0,2,22.0,POINT (-2.36161 47.27605)
2,0,2,22.0,POINT (-2.36161 47.27605)
3,0,2,22.0,POINT (-2.36161 47.27605)
4,1,6,22.0,POINT (-2.40913 47.28403)


# Bornes par zone IRIS

In [8]:
gdf_iris = get_iris_contours_city(file_iris_contours, code_insee)
gdf_iris.head()

,code_iris,nom_iris,geometry
0,440550101,La Baule les Pins,"MULTIPOLYGON (((-2.35133 47.27244, -2.35149 47..."
1,440550102,Centre-Benoît,"MULTIPOLYGON (((-2.38127 47.27993, -2.38559 47..."
2,440550103,Gare-Grand Clos,"MULTIPOLYGON (((-2.4273 47.27833, -2.42724 47...."
3,440550104,Beslon,"MULTIPOLYGON (((-2.40491 47.28723, -2.40464 47..."
4,440550105,Escoublac,"MULTIPOLYGON (((-2.38977 47.30009, -2.38918 47..."


### Detailed version: nb of charging points per power value

In [9]:
import geopandas as gpd

# Perform a spatial join to associate each charging station with its corresponding IRIS polygon
joined_gdf = gpd.sjoin(gdf_bornes_city, gdf_iris, how="inner", predicate="within")

# Group by 'code_iris', 'puissance_nominale', and 'bornes_vinci', summing 'nbre_pdc'
count_df = (
    joined_gdf.groupby(['code_iris', 'puissance_nominale', 'bornes_vinci'])['nbre_pdc']
    .sum()
    .reset_index()
)

# Pivot the table to create separate columns for each (puissance_nominale, bornes_vinci) combination
final_df = count_df.pivot(index="code_iris", columns=['puissance_nominale', 'bornes_vinci'], values="nbre_pdc").fillna(0)

# Rename columns to indicate power level and Vinci distinction
final_df.columns = [
    f"vinci_nb_pdc_{int(power)}kW" if vinci else f"concu_nb_pdc_{int(power)}kW" 
    for power, vinci in final_df.columns
]
# Merge back with the original gdf_iris to maintain all IR
final_df

,concu_nb_pdc_22kW,vinci_nb_pdc_22kW,vinci_nb_pdc_60kW,vinci_nb_pdc_120kW,concu_nb_pdc_7kW
code_iris,,,,,
440550101,8.0,132.0,48.0,0.0,0.0
440550102,42.0,384.0,324.0,12.0,0.0
440550103,0.0,342.0,108.0,0.0,0.0
440550104,0.0,144.0,0.0,0.0,2.0
440550105,0.0,144.0,0.0,0.0,0.0
440550106,0.0,144.0,0.0,0.0,0.0


### Short version: nb of charging station without power information

In [10]:
# Spatial join to assign each charging station to an IRIS code
gdf_bornes_iris = gpd.sjoin(gdf_bornes_city, gdf_iris, how="inner", predicate="within")

# Group by IRIS code and Vinci status, summing the number of charging points
summary_iris_df = (
    gdf_bornes_iris.groupby(["code_iris", "bornes_vinci"])["nbre_pdc"]
    .sum()
    .unstack(fill_value=0)  # Fill missing values with 0
    .reset_index()
)

# Rename columns to match the required format
summary_iris_df = summary_iris_df.rename(columns={0: "concu_nb_pdc", 1: "vinci_nb_pdc"})

# Merge the total charging points with the original 'gdf_iris' dataframe
final_iris_df = pd.merge(gdf_iris, summary_iris_df, how="left", on="code_iris")

# Fill NaN values in the merged DataFrame with 0 (for the charging points columns)
final_iris_df.fillna({'concu_nb_pdc': 0, 'vinci_nb_pdc': 0}, inplace=True)

# Add a column for total number of charging points
final_iris_df['nb_pdc'] = final_iris_df['concu_nb_pdc'] + final_iris_df['vinci_nb_pdc'] 

# Ensure the final DataFrame has the correct geometry
final_iris_df = gpd.GeoDataFrame(final_iris_df, geometry='geometry', crs=gdf_iris.crs)

# Display the final DataFrame
final_iris_df.head()


,code_iris,nom_iris,geometry,concu_nb_pdc,vinci_nb_pdc,nb_pdc
0,440550101,La Baule les Pins,"MULTIPOLYGON (((-2.35133 47.27244, -2.35149 47...",8,180,188
1,440550102,Centre-Benoît,"MULTIPOLYGON (((-2.38127 47.27993, -2.38559 47...",42,720,762
2,440550103,Gare-Grand Clos,"MULTIPOLYGON (((-2.4273 47.27833, -2.42724 47....",0,450,450
3,440550104,Beslon,"MULTIPOLYGON (((-2.40491 47.28723, -2.40464 47...",2,144,146
4,440550105,Escoublac,"MULTIPOLYGON (((-2.38977 47.30009, -2.38918 47...",0,144,144


In [11]:
print(final_iris_df.vinci_nb_pdc.sum())
print(final_iris_df.concu_nb_pdc.sum())

1782
52


In [12]:
# final_iris_df.to_csv(export_file_name_iris, index=False)
final_iris_df.to_file(export_file_name_iris, driver="GPKG")

In [13]:
final_iris_df = gpd.read_file(export_file_name_iris)

In [14]:
import branca

m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Add the city boundary contour
folium.GeoJson(
    geometry_contours_city,
    name="Ville",
    style_function=lambda x: {"color": "black", "weight": 2, "fillOpacity": 0.1}
).add_to(m)

# Create a colormap based on the number of people in each carreau
min_pdc = final_iris_df['nb_pdc'].min()  # Minimum number of people
max_pdc = final_iris_df['nb_pdc'].max()  # Maximum number of people
colormap = branca.colormap.linear.YlOrRd_09.scale(min_pdc, max_pdc)  # Yellow to red color scale

# Add the colormap legend to the map
colormap.add_to(m)

# Iterate through each row in the GeoDataFrame and plot each feature
for _, row in final_iris_df.iterrows():
    # Extract geometry and properties for each feature
    feature = row['geometry']
    nb_pdc = row['nb_pdc']  # Assuming this column exists
  
    # Create a popup displaying the number of people
    text = f"Number of charging points: {nb_pdc}"
    popup = folium.Popup(text, max_width=300)
        
    # Style each feature based on the number of people
    folium.GeoJson(
        feature,
        name=f"Carreau {row.nom_iris}",
        popup=popup,
        style_function=lambda x, nb_pdc=nb_pdc: {
                'fillColor': colormap(nb_pdc),  # Color based on the number of people
                'color': 'black',  # Border color
                'weight': 2,  # Border width
                'fillOpacity': 0.7  # Fill opacity
            }
    ).add_to(m)

# Return the map to visualize
m

## INSEE 200

In [15]:
gdf_carroye_city_200 = get_data_carroyee_city(file_carroye_200, geometry_contours_city, crs=crs_angles)

In [16]:
cols = ['Idcar_200m', 'geometry']
gdf_carroye_city_200 = gdf_carroye_city_200[cols]
gdf_carroye_city_200 = gdf_carroye_city_200.rename(columns={'Idcar_200m':'id'}) 

In [17]:
gdf_carroye_city_200.head()

,id,geometry
0,CRS3035RES200mN2761400E3391600,"POLYGON ((-2.3372 47.26715, -2.33762 47.26892,..."
1,CRS3035RES200mN2761400E3391800,"POLYGON ((-2.33459 47.26745, -2.33501 47.26922..."
2,CRS3035RES200mN2761400E3392000,"POLYGON ((-2.33197 47.26774, -2.3324 47.26952,..."
3,CRS3035RES200mN2761600E3391200,"POLYGON ((-2.34285 47.26832, -2.34327 47.27009..."
4,CRS3035RES200mN2761600E3391400,"POLYGON ((-2.34023 47.26862, -2.34066 47.27039..."


In [18]:
# Perform the spatial join between charging points and the insee DataFrame
gdf_bornes_insee_200 = gpd.sjoin(gdf_bornes_city, gdf_carroye_city_200, how="inner", predicate="within")

# Group by 'id' and 'bornes_vinci' to sum the number of charging points
summary_insee_df_200 = (
    gdf_bornes_insee_200.groupby(["id", "bornes_vinci"])["nbre_pdc"]
    .sum()
    .unstack(fill_value=0)  # Fill missing values with 0
    .reset_index()
)

# Rename columns to match the required format
summary_insee_df_200 = summary_insee_df_200.rename(columns={0: "concu_nb_pdc", 1: "vinci_nb_pdc"})

# Merge the total charging points with the original 'insee' dataframe
final_insee_df_200 = pd.merge(gdf_carroye_city_200, summary_insee_df_200, how="left", on="id")

# Fill NaN values in the merged DataFrame with 0 (for the charging points columns)
final_insee_df_200.fillna({'concu_nb_pdc': 0, 'vinci_nb_pdc': 0}, inplace=True)

final_insee_df_200['nb_pdc'] = final_insee_df_200['concu_nb_pdc'] + final_insee_df_200['vinci_nb_pdc'] 

final_insee_df_200 = gpd.GeoDataFrame(final_insee_df_200, geometry='geometry', crs=crs_angles)

# Display the final DataFrame
final_insee_df_200.head()


,id,geometry,concu_nb_pdc,vinci_nb_pdc,nb_pdc
0,CRS3035RES200mN2761400E3391600,"POLYGON ((-2.3372 47.26715, -2.33762 47.26892,...",0.0,0.0,0.0
1,CRS3035RES200mN2761400E3391800,"POLYGON ((-2.33459 47.26745, -2.33501 47.26922...",0.0,0.0,0.0
2,CRS3035RES200mN2761400E3392000,"POLYGON ((-2.33197 47.26774, -2.3324 47.26952,...",0.0,0.0,0.0
3,CRS3035RES200mN2761600E3391200,"POLYGON ((-2.34285 47.26832, -2.34327 47.27009...",0.0,0.0,0.0
4,CRS3035RES200mN2761600E3391400,"POLYGON ((-2.34023 47.26862, -2.34066 47.27039...",0.0,0.0,0.0


In [19]:
# Comme les carreaux ne couvrent pas toute la ville, on n'a pas toutes les bornes
print(final_insee_df_200.vinci_nb_pdc.sum()) # on perd 90 stations
print(final_insee_df_200.concu_nb_pdc.sum()) # on perd 2 stations

1692.0
50.0


In [20]:
# final_insee_df_200.to_csv(export_file_name_200, index=False)
final_insee_df_200.to_file(export_file_name_200, driver="GPKG")

In [21]:
# Load the saved DataFrame from the CSV file
# final_insee_df_1km = pd.read_csv(export_file_name_1km)
final_insee_df_200 = gpd.read_file(export_file_name_200)

# Display the first few rows to check if the data is loaded correctly
final_insee_df_200.head()

,id,concu_nb_pdc,vinci_nb_pdc,nb_pdc,geometry
0,CRS3035RES200mN2761400E3391600,0.0,0.0,0.0,"POLYGON ((-2.3372 47.26715, -2.33762 47.26892,..."
1,CRS3035RES200mN2761400E3391800,0.0,0.0,0.0,"POLYGON ((-2.33459 47.26745, -2.33501 47.26922..."
2,CRS3035RES200mN2761400E3392000,0.0,0.0,0.0,"POLYGON ((-2.33197 47.26774, -2.3324 47.26952,..."
3,CRS3035RES200mN2761600E3391200,0.0,0.0,0.0,"POLYGON ((-2.34285 47.26832, -2.34327 47.27009..."
4,CRS3035RES200mN2761600E3391400,0.0,0.0,0.0,"POLYGON ((-2.34023 47.26862, -2.34066 47.27039..."


In [22]:
import branca

m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Add the city boundary contour
folium.GeoJson(
    geometry_contours_city,
    name="Ville",
    style_function=lambda x: {"color": "black", "weight": 2, "fillOpacity": 0.1}
).add_to(m)

# Create a colormap based on the number of people in each carreau
min_pdc = final_insee_df_200['nb_pdc'].min()  # Minimum number of people
max_pdc = final_insee_df_200['nb_pdc'].max()  # Maximum number of people
colormap = branca.colormap.linear.YlOrRd_09.scale(min_pdc, max_pdc)  # Yellow to red color scale

# Add the colormap legend to the map
colormap.add_to(m)

# Iterate through each row in the GeoDataFrame and plot each feature
for _, row in final_insee_df_200.iterrows():
    # Extract geometry and properties for each feature
    feature = row['geometry']
    nb_pdc = row['nb_pdc']  # Assuming this column exists
  
    # Create a popup displaying the number of people
    text = f"Number of charging points: {nb_pdc}"
    popup = folium.Popup(text, max_width=300)
        
    # Style each feature based on the number of people
    folium.GeoJson(
        feature,
        name=f"Carreau {row.id}",
        popup=popup,
        style_function=lambda x, nb_pdc=nb_pdc: {
                'fillColor': colormap(nb_pdc),  # Color based on the number of people
                'color': 'black',  # Border color
                'weight': 2,  # Border width
                'fillOpacity': 0.7  # Fill opacity
            }
    ).add_to(m)

# Return the map to visualize
m

# Carreaux 1km

In [23]:
gdf_carroye_city_1km = get_data_carroyee_city(file_carroye_1km, geometry_contours_city, crs=crs_angles)

In [24]:
cols = ['idcar_1km', 'geometry']
gdf_carroye_city_1km = gdf_carroye_city_1km[cols]
gdf_carroye_city_1km = gdf_carroye_city_1km.rename(columns={'idcar_1km':'id'}) 

In [25]:
# Perform the spatial join between charging points and the insee DataFrame
gdf_bornes_insee_1km = gpd.sjoin(gdf_bornes_city, gdf_carroye_city_1km, how="inner", predicate="within")

# Group by 'id' and 'bornes_vinci' to sum the number of charging points
summary_insee_df_1km = (
    gdf_bornes_insee_1km.groupby(["id", "bornes_vinci"])["nbre_pdc"]
    .sum()
    .unstack(fill_value=0)  # Fill missing values with 0
    .reset_index()
)

# Rename columns to match the required format
summary_insee_df_1km = summary_insee_df_1km.rename(columns={0: "concu_nb_pdc", 1: "vinci_nb_pdc"})

# Merge the total charging points with the original 'insee' dataframe
final_insee_df_1km = pd.merge(gdf_carroye_city_1km, summary_insee_df_1km, how="left", on="id")

# Fill NaN values in the merged DataFrame with 0 (for the charging points columns)
final_insee_df_1km.fillna({'concu_nb_pdc': 0, 'vinci_nb_pdc': 0}, inplace=True)


final_insee_df_1km['nb_pdc'] = final_insee_df_1km['concu_nb_pdc'] + final_insee_df_1km['vinci_nb_pdc'] 

# Display the final DataFrame
final_insee_df_1km = gpd.GeoDataFrame(final_insee_df_1km, geometry='geometry', crs=crs_angles)



In [26]:
# on récupère bien toutes les stations
print(final_insee_df_1km.vinci_nb_pdc.sum())
print(final_insee_df_1km.concu_nb_pdc.sum())

1782.0
52.0


In [27]:
# final_insee_df_1km.to_csv(export_file_name_1km, index=False)
final_insee_df_1km.to_file(export_file_name_1km, driver="GPKG")

In [28]:
final_insee_df_1km = gpd.read_file(export_file_name_1km)

# Display the first few rows to check if the data is loaded correctly
final_insee_df_1km.head()

,id,concu_nb_pdc,vinci_nb_pdc,nb_pdc,geometry
0,CRS3035RES1000mN2761000E3390000,0.0,0.0,0.0,"POLYGON ((-2.35724 47.26121, -2.35937 47.27007..."
1,CRS3035RES1000mN2761000E3391000,0.0,72.0,72.0,"POLYGON ((-2.34418 47.26271, -2.34631 47.27156..."
2,CRS3035RES1000mN2761000E3392000,0.0,0.0,0.0,"POLYGON ((-2.33112 47.2642, -2.33325 47.27306,..."
3,CRS3035RES1000mN2761000E3393000,0.0,0.0,0.0,"POLYGON ((-2.31807 47.26569, -2.32019 47.27455..."
4,CRS3035RES1000mN2762000E3389000,8.0,0.0,8.0,"POLYGON ((-2.37243 47.26857, -2.37456 47.27743..."


In [29]:
import branca

m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Add the city boundary contour
folium.GeoJson(
    geometry_contours_city,
    name="Ville",
    style_function=lambda x: {"color": "black", "weight": 2, "fillOpacity": 0.1}
).add_to(m)

# Create a colormap based on the number of people in each carreau
min_pdc = final_insee_df_1km['nb_pdc'].min()  # Minimum number of people
max_pdc = final_insee_df_1km['nb_pdc'].max()  # Maximum number of people
colormap = branca.colormap.linear.YlOrRd_09.scale(min_pdc, max_pdc)  # Yellow to red color scale

# Add the colormap legend to the map
colormap.add_to(m)

# Iterate through each row in the GeoDataFrame and plot each feature
for _, row in final_insee_df_1km.iterrows():
    # Extract geometry and properties for each feature
    feature = row['geometry']
    nb_pdc = row['nb_pdc']  # Assuming this column exists
  
    # Create a popup displaying the number of people
    text = f"Number of charging points: {nb_pdc}"
    popup = folium.Popup(text, max_width=300)
    
    # print(num_people)
    

    # Style each feature based on the number of people
    folium.GeoJson(
        feature,
        name=f"Carreau {row.id}",
        popup=popup,
        style_function=lambda x, nb_pdc=nb_pdc: {
                'fillColor': colormap(nb_pdc),  # Color based on the number of people
                'color': 'black',  # Border color
                'weight': 2,  # Border width
                'fillOpacity': 0.7  # Fill opacity
            }
    ).add_to(m)

# Return the map to visualize
m